In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier

data = pd.read_csv('pseudodata_præoperation.csv')
y = np.random.choice([0,1],size = len(data))

X = data.to_numpy()

In [5]:
num_negative = np.sum(y == 0)
num_positive = np.sum(y == 1)
scale_pos_weight = num_negative / num_positive
scale_pos_weight

0.9721362229102167

In [ ]:
weights = [scale_pos_weight*0.25,scale_pos_weight*0.5,scale_pos_weight*0.75,scale_pos_weight,scale_pos_weight*1.75,scale_pos_weight*1.5,scale_pos_weight*2]

In [ ]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
from skopt.space import Real, Integer

# # Generate synthetic data
# X, y = make_classification(n_samples=6000, n_features=20, random_state=42)

# Split data into training and hold-out sets
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=1/6, random_state=42)

# Calculate scale_pos_weight based on training data
num_negative = np.sum(y_train == 0)
num_positive = np.sum(y_train == 1)
scale_pos_weight = num_negative / num_positive

# Initialize XGBoost classifier
model = XGBClassifier()

# Define the hyperparameter space
param_space = {
    'n_estimators': Integer(10, 5000),
    'max_depth': Integer(1, 10),
    'learning_rate': Real(0.001, 0.01, prior='log-uniform'),
    'subsample': Real(0.6, 1.0),
    'colsample_bytree': Real(0.6, 1.0),
    'gamma': Real(0, 5),
    'scale_pos_weight': Real(scale_pos_weight*0.25, scale_pos_weight*2)
}

# Define the scoring metrics
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# Initialize BayesSearchCV
opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    scoring=scoring,
    n_iter=50,  # Number of iterations for Bayesian optimization
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,
    return_train_score=True,
    refit='roc_auc',  # Refit the best model on the entire training set using ROC AUC
    random_state=42
)

# Perform Bayesian optimization
opt.fit(X_train, y_train)

# Evaluate on hold-out set
y_pred_holdout = opt.predict(X_holdout)
print("Hold-Out Set Metrics:")
for metric in scoring:
    if metric in ['accuracy', 'precision', 'recall', 'f1']:
        score = globals()[f"{metric}_score"](y_holdout, y_pred_holdout)
    elif metric == 'roc_auc':
        score = roc_auc_score(y_holdout, opt.predict_proba(X_holdout)[:, 1])
    print(f"{metric.capitalize()}: {score}")

# Print the best parameters and the best score
print("Best parameters:", opt.best_params_)
print("Best ROC AUC:", opt.best_score_)


In [4]:
Real(scale_pos_weight*0.25, scale_pos_weight*2)

Real(low=0.25, high=2.0, prior='uniform', transform='identity')

In [ ]:
from xgboost import XGBClassifier
from skopt.space import Real, Integer
from skopt import BayesSearchCV

# Initialize the classifier with learning rate
model = XGBClassifier(learning_rate=0.1, max_depth=6, n_estimators=100)

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# Define the hyperparameter space
param_space = {
    'n_estimators': Integer(10, 100),
    'max_depth': Integer(1, 20),
    'learning_rate': Real(0.001, 0.01, prior='log-uniform'),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 20)
}

# Define the scoring metrics
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# Initialize BayesSearchCV
opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    scoring='roc_auc',  # This is the primary scoring metric for optimization
    n_iter=50,
    cv=5,
    n_jobs=-1,
    return_train_score=True,
    refit='roc_auc'
)

# Perform the optimization
opt.fit(X, y)

# Print the best parameters and the best score
print("Best parameters:", opt.best_params_)
print("Best ROC AUC:", opt.best_score_)

# Optionally, evaluate other metrics using cross_validate
best_model = opt.best_estimator_
additional_scores = ['accuracy', 'precision', 'recall', 'f1']
cross_val_scores = cross_validate(best_model, X, y, scoring=additional_scores, cv=5, n_jobs=-1, return_train_score=True)


scores = cross_validate(model, X, y, scoring=scoring, cv=5, n_jobs=-1, return_train_score=True)

for metric in scoring:
    print(f"{metric.capitalize()}: {scores['test_' + metric].mean()}")

for metric in scoring:
    print(f"Train {metric.capitalize()}: {scores['train_' + metric].mean()}")



In [ ]:
# evaluate xgboost algorithm for classification
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
# define dataset
# data = pd.read_csv('x_matricer/x_matrix_pre_4729.csv', compression='gzip').sort_values(by='IDno',ignore_index=True).drop(columns='IDno')
# y_pred = pd.read_csv('y_pred_4729.csv').sort_values(by='IDno',ignore_index=True).drop(columns='IDno')

data = pd.read_csv('pseudodata_præoperationer.csv', compression='gzip').sort_values(by='IDno',ignore_index=True).drop(columns='IDno')
y = np.random.choice([0,1],size = len(data))

# Convert to numpy array and generate synthetic labels for demonstration
X = data.to_numpy()
# y = y_pred.to_numpy()
# define the model
model = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring = 'accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
from xgboost import XGBRegressor
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model = XGBRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
# explore xgboost number of trees effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from matplotlib import pyplot



# get a list of models to evaluate
def get_models():
	models = dict()
	trees = [10, 50, 100, 500, 1000, 5000]
	for n in trees:
		models[str(n)] = XGBClassifier(n_estimators=n)
	return models

# evaluate a give model using cross-validation
def evaluate_model(model):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# explore xgboost tree depth effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from matplotlib import pyplot


# get a list of models to evaluate
def get_models():
	models = dict()
	for i in range(1,11):
		models[str(i)] = XGBClassifier(max_depth=i)
	return models

# evaluate a give model using cross-validation
def evaluate_model(model):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# explore xgboost learning rate effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	rates = [0.00001, 0.001, 0.01, 0.1, 1.0]
	for i in range(1,11):
		for r in rates:
			key = '%.4f' % r
			models[(str(i),key)] = XGBClassifier(max_depth=i,eta=r)
	return models

# evaluate a give model using cross-validation
def evaluate_model(model):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# explore xgboost subsample ratio effect on performance
from numpy import arange
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	for i in arange(0.1, 1.1, 0.1):
		key = '%.1f' % i
		models[key] = XGBClassifier(subsample=i)
	return models

# evaluate a give model using cross-validation
def evaluate_model(model):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores


# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	print(name)
	scores = evaluate_model(model)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [7]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score

# Load and preprocess data
# data = pd.read_csv('x_matricer/x_matrix_pre_4729.csv', compression='gzip').sort_values(by='IDno', ignore_index=True).drop(columns='IDno')
# y_pred = pd.read_csv('y_pred_4729.csv').sort_values(by='IDno', ignore_index=True).drop(columns='IDno')
# X = data.to_numpy()
# y = y_pred.to_numpy().flatten()

# # Standardize the data
# X = StandardScaler().fit_transform(X)

data = pd.read_csv('pseudodata_præoperation.csv')
y = np.random.choice([0,1],size = len(data))

X = data.to_numpy()

# Split data into holdout set and remaining set
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=1/6, random_state=42)

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dholdout = xgb.DMatrix(X_holdout, label=y_holdout)

scale_pos_weight = num_negative / num_positive

# Define a simple parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.0001, 0.001, 0.01, 0.3],
    'n_estimators': [10, 50, 100, 500, 1000, 5000],
    'scale_pos_weight': [0.1*num_negative,num_negative,num_negative*1.5,num_negative*2]  # Adjust for class imbalance
}

# Initialize a XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False)

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

# Perform Grid Search with Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters found
print(f'Best parameters found: {grid_search.best_params_}')
print(f'Best cross-validation accuracy: {grid_search.best_score_:.5f}')

# Train the final model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the final model on the holdout set
y_holdout_pred = best_model.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)

print(f'Final Model Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'Holdout Set Precision: {precision * 100:.2f}%')
print(f'Holdout Set Recall: {recall * 100:.2f}%')


Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best parameters found: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 1000, 'scale_pos_weight': 1327}
Best cross-validation accuracy: 0.50113
Final Model Accuracy on Holdout Set: 50.47%
Holdout Set Precision: 48.66%
Holdout Set Recall: 79.37%


In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix

# Split data into holdout set and remaining set
train_indices, holdout_indices = train_test_split(np.arange(len(X)), test_size=500, random_state=42, stratify=y)
X_train, X_holdout = X[train_indices], X[holdout_indices]
y_train, y_holdout = y[train_indices], y[holdout_indices]

# Check class distribution
print("Class distribution in training set:", np.bincount(y_train.astype(int)))
print("Class distribution in holdout set:", np.bincount(y_holdout.astype(int)))

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dholdout = xgb.DMatrix(X_holdout, label=y_holdout)

# Define a simple parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [np.sum(y_train == 0) / np.sum(y_train == 1)]  # Adjust for class imbalance
}

# Initialize a XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')

# Perform Grid Search with Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters found
print(f'Best parameters found: {grid_search.best_params_}')
print(f'Best cross-validation accuracy: {grid_search.best_score_:.5f}')

# Train the final model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the final model on the holdout set
y_holdout_pred = best_model.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)

print(f'Final Model Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'Holdout Set Precision: {precision * 100:.2f}%')
print(f'Holdout Set Recall: {recall * 100:.2f}%')

# Confusion Matrix
cm = confusion_matrix(y_holdout, y_holdout_pred)
print("Confusion Matrix:")
print(cm)


In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix, roc_auc_score


# Split data into holdout set and remaining set
train_indices, holdout_indices = train_test_split(np.arange(len(X)), test_size=500, random_state=42, stratify=y)
X_train, X_holdout = X[train_indices], X[holdout_indices]
y_train, y_holdout = y[train_indices], y[holdout_indices]

# Check class distribution
print("Class distribution in training set:", np.bincount(y_train.astype(int)))
print("Class distribution in holdout set:", np.bincount(y_holdout.astype(int)))

# Initialize a simple XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss', scale_pos_weight=np.sum(y_train == 0) / np.sum(y_train == 1))

# Perform Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation accuracy: {cv_scores.mean():.5f}')

# Train the final model on the entire training set
xgb_model.fit(X_train, y_train)

# Evaluate the final model on the holdout set
y_holdout_pred = xgb_model.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)
roc_auc = roc_auc_score(y_holdout, y_holdout_pred)

print(f'Final Model Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'Holdout Set Precision: {precision * 100:.2f}%')
print(f'Holdout Set Recall: {recall * 100:.2f}%')
print(f'Holdout Set ROC AUC: {roc_auc * 100:.2f}%')

# Confusion Matrix
cm = confusion_matrix(y_holdout, y_holdout_pred)
print("Confusion Matrix:")
print(cm)


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Train the model to inspect feature importance
xgb_model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss', scale_pos_weight=np.sum(y_train == 0) / np.sum(y_train == 1))
xgb_model.fit(X_train, y_train)

# Plot feature importance
xgb.plot_importance(xgb_model, max_num_features=20)
plt.show()


In [ ]:
# Inspect random samples
sample_indices = np.random.choice(train_indices, size=5, replace=False)
print(data.iloc[sample_indices])
print(y_pred.iloc[sample_indices])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix

# Train logistic regression
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
logreg.fit(X_train, y_train)

# Evaluate on holdout set
y_holdout_pred = logreg.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)
roc_auc = roc_auc_score(y_holdout, y_holdout_pred)

print(f'Logistic Regression Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'Logistic Regression Precision: {precision * 100:.2f}%')
print(f'Logistic Regression Recall: {recall * 100:.2f}%')
print(f'Logistic Regression ROC AUC: {roc_auc * 100:.2f}%')

# Confusion Matrix
cm = confusion_matrix(y_holdout, y_holdout_pred)
print("Confusion Matrix:")
print(cm)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate on holdout set
y_holdout_pred = rf.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)
roc_auc = roc_auc_score(y_holdout, y_holdout_pred)

print(f'Random Forest Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'Random Forest Precision: {precision * 100:.2f}%')
print(f'Random Forest Recall: {recall * 100:.2f}%')
print(f'Random Forest ROC AUC: {roc_auc * 100:.2f}%')

# Confusion Matrix
cm = confusion_matrix(y_holdout, y_holdout_pred)
print("Confusion Matrix:")
print(cm)


In [ ]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Train XGBoost on augmented data
xgb_model.fit(X_train_res, y_train_res)

# Evaluate on holdout set
y_holdout_pred = xgb_model.predict(X_holdout)
accuracy = accuracy_score(y_holdout, y_holdout_pred)
precision = precision_score(y_holdout, y_holdout_pred)
recall = recall_score(y_holdout, y_holdout_pred)
roc_auc = roc_auc_score(y_holdout, y_holdout_pred)

print(f'SMOTE XGBoost Accuracy on Holdout Set: {accuracy * 100:.2f}%')
print(f'SMOTE XGBoost Precision: {precision * 100:.2f}%')
print(f'SMOTE XGBoost Recall: {recall * 100:.2f}%')
print(f'SMOTE XGBoost ROC AUC: {roc_auc * 100:.2f}%')

# Confusion Matrix
cm = confusion_matrix(y_holdout, y_holdout_pred)
print("Confusion Matrix:")
print(cm)
